In [ ]:
%pip install kaggle -q

In [ ]:
!kaggle -v

Kaggle API 1.6.11


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!mkdir -p data
!kaggle datasets download -d azathoth42/myanimelist -p data
!unzip -o data/myanimelist.zip -d data

1409.81s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1415.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Dataset URL: https://www.kaggle.com/datasets/azathoth42/myanimelist
License(s): DbCL-1.0
100%|█████████████████████████████████████▉| 1.92G/1.92G [04:23<00:00, 10.1MB/s]
100%|██████████████████████████████████████| 1.92G/1.92G [04:23<00:00, 7.84MB/s]


1685.61s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Archive:  data/myanimelist.zip
  inflating: data/AnimeList.csv      
  inflating: data/UserAnimeList.csv  
  inflating: data/UserList.csv       
  inflating: data/anime_cleaned.csv  
  inflating: data/anime_filtered.csv  
  inflating: data/animelists_cleaned.csv  
  inflating: data/animelists_filtered.csv  
  inflating: data/users_cleaned.csv  
  inflating: data/users_filtered.csv  


In [8]:
import pandas as pd

anime = pd.read_csv("data/AnimeList.csv")
users = pd.read_csv("data/users_cleaned.csv")
ratings = pd.read_csv("data/animelists_cleaned.csv")